<a href="https://colab.research.google.com/github/bommojuvishu/facebook-prophet-stock/blob/main/Copy_of_fbphrophit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 9.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [55]:
from datetime import date ,datetime ,timedelta
import yfinance as yf
from fbprophet import Prophet
import operator
import pandas as pd
import io
import traceback


In [ ]:
def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

In [ ]:
def cal_percentage(data):
  x= data['yhat'].tolist()
  y = x[-1] -x[0]
  y=y/x[0]
  y = y *100
  return y

In [ ]:
# Calculate the percentage returns
def cal_real_perent(stock, STARTDATE, ENDDATE):
  try:
    data = yf.download(stock +'.NS', STARTDATE, ENDDATE)
    data.reset_index(inplace=True)
    openprice=data['Open'].tolist()[0]
    closeprice = data['Close'].tolist()[-1]
    x= closeprice - openprice
    x= x / openprice
    x = x*100
    return x 
  except:
    print("ERROR Real data: ", stock)
    traceback.print_exc()
    return 0 

In [ ]:
import pandas as pd 
import yfinance as yf
import matplotlib.pyplot as plt
import datetime as dt

start=dt.datetime.today()-dt.timedelta(160)
end=dt.datetime.today()
clprice=pd.DataFrame(yf.download("FB MSFT AMZN GOOG GM", start=start, end=end))
clprice=clprice.dropna()
def macd(DF,a,b,c):
  df=DF.copy()
  df['MA Fast']=df['Adj Close'].ewm(span=a, min_periods=a).mean()
  df['MA Slow']=df['Adj Close'].ewm(span=b, min_periods=b).mean()
  df["MACD"]=df['MA Fast']-df['MA Slow']
  df['Signal']=df.MACD.ewm(span=c, min_periods=c).mean()
  df["Histrogram"]=df.MACD-df.Signal
  df=df.dropna()
  
  return df["Histrogram"].tolist()[-1]

[*********************100%***********************]  5 of 5 completed


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving nifty500.csv to nifty500.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['nifty500.csv']))
print(df)

        3MINDIA
0           ABB
1           ACC
2        AIAENG
3     APLAPOLLO
4        AUBANK
..          ...
495     YESBANK
496        ZEEL
497  ZENSARTECH
498   ZYDUSWELL
499      ECLERX

[500 rows x 1 columns]


In [64]:
stocks =['TEchM','POWERGRID']
len(stocks)

2

In [65]:
START = "2015-01-01"
TODAY = "2021-12-02"
period = 30

datetime_object = datetime.strptime(TODAY, '%Y-%m-%d')
datetime_object =datetime_object + timedelta(30)
STARTDATE = TODAY
ENDDATE = datetime_object.strftime("%Y-%m-%d")
# TODAY = date.today().strftime("%Y-%m-%d")

result={}

error_stocks=[]
# stocks = df['3MINDIA'].tolist()

# stocks = ('TTML','HGS', 'INFY')

for stock in stocks:
  try:
    allRts = []
    data = load_data(stock +".NS")
    hista =macd(data, 12,26,9)
    df_train = data[['Date','Close']]
    df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})
    df_train.tail()
    data_rows = data.shape[0]
    m = Prophet()
    m.fit(df_train)
    future = m.make_future_dataframe(periods=period)
    forecast = m.predict(future)
    predict_rows = forecast.shape[0]
    forecasted = forecast.tail(predict_rows - data_rows)
    forecast_rtr =  cal_percentage(forecasted)
    # Cal Real returns 
    realrtr = cal_real_perent(stock, STARTDATE, ENDDATE)
    
    allRts.append(forecast_rtr)
    allRts.append(realrtr)
    allRts.append(hista)
    print(stock)

    result[stock] =allRts

  except Exception as e:
    print("ERROR :", stock  )
    
    error_stocks.append(stock)
    traceback.print_exc()

[*********************100%***********************]  1 of 1 completed

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



[*********************100%***********************]  1 of 1 completed
TEchM
[*********************100%***********************]  1 of 1 completed

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



[*********************100%***********************]  1 of 1 completed
POWERGRID


In [66]:
resultdf = pd.DataFrame(result)
resultdf = resultdf.transpose()
resultdf.columns =['predict','real','hista']
print(resultdf.sort_values(by=['real'], ascending=False).head(50))

            predict       real     hista
TEchM      2.919989  12.613211 -4.911801
POWERGRID  2.147003  -2.200960  1.658606


In [ ]:
resultdf.to_csv('nifty500predict.csv')

In [68]:
len(forecasted)

30